# LangGraph Autonomous Panel of Experts Framework
New: 
1) Chain of thought reasoning to agents in their prompt
2) Response aggregator
3) User rating system

This notebook shows a stateful llm multi-agent application that creates and coordinates AI agents to solve a problem while iterativly improving results using reflection.

1. Ideation:
   - Analyzes the user's problem statement
   - Create necessary AI agents to solve the problem
   - <span style="color:Gold">*State: Summary of what agents were made and why + analysis of the problem for deeper understanding*</span>

2. Delegation:
   - Generate and assign a description and list of tasks for each agent
   - <span style="color:Gold">*State: "Agent delegration phase properly ran" <-- todo: need to include later*</span>
3. Execution & Synthesis:
   - Use a LLM to run each agent given their description, name, and assigned tasks
   - Aggregate responses from all agents and consolidates "best" approaches and presents to user for rating
   - <span style="color:Gold">*State: Responses from each agent will be appended to the state*</span>

4. Evaluation:
   - Assess solution quality and its alignment with user prompt
   - User gives a rating from 1-5 based on accuracy, relevance, and completeness
   - Identify steps to follow for improvement in next iteration
   - <span style="color:Gold">*State: An evauluation of the solution quality and next steps to implement*</span>

In [1]:
#!pip install -U --quiet langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph grandalf

## Load model

In [2]:
from langchain_openai.chat_models import ChatOpenAI
# llm = ChatOpenAI(temperature=0, model_name="mixtral-8x7b-instruct", openai_api_key="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", base_url="https://api.perplexity.ai")

# llm = ChatOpenAI(temperature=0.0, base_url="http://localhost:1234/v1", api_key="not-needed")
llm = ChatOpenAI(base_url="http://rjain-40832s.nrel.gov:1234/v1", api_key="lm-studio")

In [3]:
# llm.invoke("What model are you?").content

## Define the problem statement

In [4]:
main_prompt =  """
You have access to the following data:

- Consumer load:
    - 2 kW from 12am to 7am
    - 4 kW from 7am to 9am
    - 5 kW from 9am to 3pm
    - 2 kW from 3pm to 7pm
    - 4 kW from 7pm to 9pm
    - 2 kW frp, 9pm tp 12am

- Electricity costs $0.1 from 12am to 7am, $0.15 from 7am to 9am, $0.2 from 9am to 3pm, $0.1 from 3pm to 7pm, $0.15 from 7pm to 9pm, and $0.1 from 9pm to 12am.

- Weather Data for San Diego, CA:
  - Temperature: 17.57°C
  - Humidity: 64%
  - Pressure: 1012 hPa
  - Description: broken clouds
- Utility demand response plan: Consumer load should be below 3 kW between 9am and 3pm


Throughout your discussion you should consider the provided data, analyze the potential impact of proposed strategies, identify risks and limitations, and suggest mitigation measures to ensure the overall effectiveness and feasibility of the plan.   

All the data you need is given already. Do not generate code."""

## Define agents for each stage

### Ideation
   - Analyzes the user's problem statement
   - Create necessary AI agents to solve the problem

In [5]:
import re
from langchain_core.messages import HumanMessage


iteration = 0
agent_profiles = {}
agent_list = []
problem_analysis = ""
generate_POE = ""



def extract_brackets(string): # this returns a list
    match = re.search(r'\[(.*?)\]', string)
    if match:
        return [item.strip() for item in match.group(1).split(',')]
    else:
        return []

def create_agent_profiles_from_list(agent_list): # this returns a dictionary
    global agent_profiles
    for i in agent_list:
        agent_profiles[i] = {
            "Description": "",
            "Tasks": ""
        }
    return agent_profiles 


def find_list_of_agents(response):
    list_finder_prompt = """
You are an AI agent list extractor. Your role is to search for the list of agent names within a given string and provide the list in a specific format.
The input string will contain the response generated by the AI agent creation assistant. Within this response, there should be a list of agent names in the following format:
[AgentName1, AgentName2, AgentName3, ...]
Your task is to:
Search for the list of agent names within the provided input string.
If the list is found, extract it exactly as it appears in the string, maintaining the square brackets and comma separation.
Respond with the extracted list of agent names in the following format:
[AgentName1, AgentName2, AgentName3, ...]
If the list is not found or is not in the expected format, respond with an empty list:
[]
Please ensure that your response only includes the list of agent names in the specified format, without any additional text, prefixes, or commentary.
Your goal is to accurately locate and extract the list of agent names from the given string and provide it in the specified format. This will help with easily identifying the agents created by the AI agent creation assistant.
Remember, focus solely on finding and providing the list of agent names, as this information is essential for the next steps in the problem-solving process.
"""
    
    raw_list = llm.invoke(f"{list_finder_prompt}\nSearch this response{response}",{"recursion_limit": 100}).content

    return extract_brackets(raw_list)


def dce_0(state):

    print(f"""
        
        STARTED ITERATION NUMBER 1
        
        """)
    messages = state["messages"]
    
    # global iteration
    # iteration += 1

    probem_analysis_prompt = """You are an AI problem analysis agent. Your role is to carefully review the user's problem statement and provide a thorough, critical analysis of the problem using chain-of-thought reasoning.

When analyzing the problem, engage in chain-of-thought reasoning by following these steps and explaining your thought process at each stage:

1. Identify the key objectives, goals, and desired outcomes stated in or implied by the problem statement.
   - Reasoning: Explain how you determined the objectives and goals based on the information provided in the problem statement.

2. Determine the main challenges, obstacles, or constraints that need to be considered when addressing the problem.
   - Reasoning: Describe your thought process for identifying the challenges and constraints, and why they are relevant to the problem.

3. Break down the problem into its core components or sub-problems, and analyze each one individually.
   - Reasoning: Explain how you decided to break down the problem into specific components, and provide a step-by-step analysis of each sub-problem.

4. Identify any assumptions, uncertainties, or ambiguities in the problem statement that may require further clarification.
   - Reasoning: Describe how you identified the assumptions and uncertainties, and explain why they are important to clarify for solving the problem.

5. Evaluate the feasibility and practicality of solving the problem, considering factors such as available resources, time constraints, and potential risks.
   - Reasoning: Explain your thought process for assessing the feasibility and practicality of solving the problem, considering the relevant factors.

6. Provide insights into the problem's complexity, potential solution approaches, and any trade-offs that may need to be considered.
   - Reasoning: Describe how you determined the problem's complexity and identified potential solution approaches, and explain your reasoning behind any trade-offs.

The output should be formatted as follows:

Problem Analysis:
- Objectives and goals: <list of objectives and desired outcomes>
  - Reasoning: <explanation of how objectives and goals were determined>
- Challenges and constraints: <list of challenges and obstacles>
  - Reasoning: <explanation of how challenges and constraints were identified>
- Problem breakdown:
   1. <sub-problem or component 1>
      - Analysis: <step-by-step analysis of sub-problem 1>
      - Reasoning: <explanation of how the problem was broken down and analyzed>
   2. <sub-problem or component 2>
      - Analysis: <step-by-step analysis of sub-problem 2>
      - Reasoning: <explanation of how the problem was broken down and analyzed>
   ...
- Assumptions and uncertainties: <list of assumptions and ambiguities>
  - Reasoning: <explanation of how assumptions and uncertainties were identified>
- Feasibility and practicality: <evaluation of feasibility and practicality>
  - Reasoning: <explanation of the thought process for assessing feasibility and practicality>
- Problem complexity and solution approaches: <insights into complexity and potential solution approaches>
  - Reasoning: <explanation of how complexity was determined and solution approaches were identified>

By utilizing chain-of-thought reasoning and providing a detailed explanation of your thought process at each step, you will deliver a comprehensive, insightful, and action-oriented problem analysis that provides a solid foundation for understanding the problem and guiding the development of effective solutions.""" # <---

 #After defining each agent, create a final list containing the names of all the agents, enclosed in square brackets and separated by commas, like this: [AgentName1, AgentName2, AgentName3, ...] '''

    global problem_analysis
    problem_analysis = llm.invoke(f"{probem_analysis_prompt} \n\n Here is the problem statement:{messages}",{"recursion_limit": 100}).content

    create_panel_of_experts_prompt = """You are an AI agent creation assistant. Your role is to design a set of AI agents that will work together to solve the problem, based on the provided problem analysis. When creating the agents, engage in chain-of-thought reasoning to clearly explain your thought process and decision-making.

For each agent, define:
1. Name: A unique, descriptive name for the agent that reflects its role or specialty.
2. Rationale: A clear explanation of why this agent is needed and how it will contribute to solving the overall problem, based on the specific objectives, challenges, and sub-problems identified in the problem analysis.
   - Reasoning: Provide a step-by-step explanation of your thought process for creating this agent, including how you determined its role and capabilities based on the problem analysis.

When creating the agents, consider the following:
- Each agent should address a specific aspect of the problem or tackle a particular sub-problem identified in the analysis.
  - Reasoning: Explain how you decided to assign specific aspects or sub-problems to each agent, ensuring that their roles are distinct and complementary.

- Agents should have distinct roles and capabilities that complement each other to cover all the necessary aspects of the problem.
  - Reasoning: Describe your thought process for ensuring that the agents' collective roles and capabilities are sufficient to address the entire problem effectively.

- The rationale for each agent should clearly link its purpose to the problem analysis, explaining how it will help overcome challenges, achieve objectives, or solve sub-problems.
  - Reasoning: Provide a detailed explanation of how you connected each agent's purpose to the specific elements of the problem analysis, demonstrating the reasoning behind your decisions.

The focus should be on providing a clear, concise, and well-reasoned explanation for why each agent is needed in the context of the problem analysis. Use chain-of-thought reasoning to walk through your decision-making process step-by-step, ensuring that your rationale is transparent and logical.

Create as many agents as you deem necessary to comprehensively address the problem, ensuring that their collective roles and capabilities are sufficient to solve the problem effectively.
- Reasoning: Explain your thought process for determining the number of agents required, considering factors such as problem complexity, sub-problem distribution, and agent specialization.

After defining each agent, provide a final list containing the names of all the agents you have created. The list should be in the following format:
[AgentName1, AgentName2, AgentName3, ...]
Each agent name should be enclosed in square brackets and separated by commas. This list should appear at the end of your response, after all the agent definitions.

Remember, the final list of agent names is crucial for the next steps in the problem-solving process, so please ensure that it is provided in the specified format.

By engaging in chain-of-thought reasoning throughout the agent creation process, you will provide a transparent and well-justified set of AI agents that are specifically designed to tackle the problem at hand, based on a thorough understanding of the problem analysis."""

    global generate_POE
    generate_POE = llm.invoke(f""" {create_panel_of_experts_prompt}
    
    Problem analysis:
    {problem_analysis}
    """).content

    global agent_list

    agent_list = find_list_of_agents(generate_POE)

    create_agent_profiles_from_list(agent_list) # <--- this updates the global agent_profiles dictionary

    # ? can also create a summary of generate_POE


    return {"messages": [HumanMessage(content=generate_POE)]}
 

# def dce_0(state):
#     '''
#     1. analyze problem
#     2. create agents
    
#     '''
#     global iteration

#     return {"messages": []}



In [6]:
# content = '''You are an AI agent creation assistant. Your role is to design a set of AI agents that will work together to solve the problem, based on the provided problem analysis.\n\nFor each agent, define:\n1. Name: A unique, descriptive name for the agent that reflects its role or specialty.\n2. Rationale: A clear explanation of why this agent is needed and how it will contribute to solving the overall problem, based on the specific objectives, challenges, and sub-problems identified in the problem analysis.\n\nWhen creating the agents, consider the following:\n- Each agent should address a specific aspect of the problem or tackle a particular sub-problem identified in the analysis.\n- Agents should have distinct roles and capabilities that complement each other to cover all the necessary aspects of the problem.\n- The rationale for each agent should clearly link its purpose to the problem analysis, explaining how it will help overcome challenges, achieve objectives, or solve sub-problems.\n\nThe focus should be on providing a clear, concise, and well-reasoned explanation for why each agent is needed in the context of the problem analysis.\n\nCreate as many agents as you deem necessary to comprehensively address the problem, ensuring that their collective roles and capabilities are sufficient to solve the problem effectively.\n\nAfter defining each agent, provide a final list containing the names of all the agents you have created. The list should be in the following format:\n[AgentName1, AgentName2, AgentName3, ...]\nEach agent name should be enclosed in square brackets and separated by commas. This list should appear at the end of your response, after all the agent definitions.\n\nRemember, the final list of agent names is crucial for the next steps in the problem-solving process, so please ensure that it is provided in the specified format.\n    \n    Problem analysis:\n    Problem Analysis:\n\n- Objectives and goals:\n   - Minimize electricity costs for the consumer.\n   - Ensure compliance with the utility demand response plan (load below 3 kW between 9am and 3pm).\n   - Optimize energy consumption based on weather conditions.\n\n- Challenges and constraints:\n   - Time-varying electricity rates and load patterns.\n   - Weather-dependent energy usage, especially for heating or cooling.\n   - Limited control over the consumer's load during specific hours (e.g., between 9am and 3pm).\n   - Potential risks of non-compliance with the demand response plan leading to penalties.\n\n- Problem breakdown:\n   1. Energy consumption optimization:\n       - Analysis: Develop a strategy that adjusts load based on time, cost, and weather data to minimize costs while adhering to the utility's requirements.\n   2. Weather impact analysis:\n       - Analysis: Assess how temperature, humidity, and pressure affect energy usage and identify potential opportunities for load shifting or energy conservation.\n   3. Compliance with demand response plan:\n       - Analysis: Design a mechanism to monitor and control load during peak hours, ensuring it remains below the 3 kW limit.\n\n- Assumptions and uncertainties:\n   - The consumer is willing to participate in load management programs.\n   - Weather data accurately predicts energy consumption patterns.\n   - No unforeseen changes in electricity rates or utility policies.\n\n- Feasibility and practicality:\n   - Feasible with access to real-time data and smart home automation systems.\n   - Practical if the consumer has some control over their appliances' usage patterns.\n\n- Problem complexity and solution approaches:\n   - Complexity lies in balancing cost savings, compliance, and comfort.\n   - Potential solutions include scheduling energy-intensive tasks during off-peak hours, using energy-efficient appliances, and implementing a dynamic pricing strategy based on weather and utility rates.\n\nFinal list: [EnergyConsumptionOptimizationAgent, WeatherImpactAnalysisAgent, DemandResponseComplianceAgent]\n'''

# x = find_list_of_agents(content)

# y = create_agent_profiles_from_list(x)

# y

In [7]:
# agent_profiles

### Delegation
   - Generate and assign a description and list of tasks for each agent
   - *State: Summary of steps taken in this phase // the input to the llm to summarize will be str(dictionary)*

   > Needs to iterpert feedback from CEA while also putting in a condition that says, if the steps don't relate to a certain agent, then no need to change antyhing // this needs to search for  a list ([...]) and assign it to a new one and iterate through it

   e.g new_agents = extract_brackets(CEA_feedback) <-- new_agents should be a local variable that's reset everytime it is called because we want it to change upon every iteration and not remain static

   for i in new_agents:
      ...

   afterwards, need to have prompt give a list like during decomposition stage

In [8]:
'''
How do these two agents update the state? There are a few ways we can go about this...
        1) Have them append the llm response to the state
        2) Have the dictionary be appended to the state (this may make description and task output be out of order, may need to consolidate it) // it'll be better to consolidate it into one

'''

def dce_1(state):

    global iteration
    
    # if iteration >= 2:
    #     print("Consider these reccomendations from the critical evaluation agent on how to improve solution quality and...")
    # else:
    #     return {"messages": " "}

    return {"messages": [HumanMessage(content="DCE_1 was properly runned")]}
        

# ? This needs to be able to know if this is a 2nd iteration or not
def generate_description_and_task(state):
    global agent_list
    global agent_profiles
    global main_prompt
    global problem_analysis
    global generate_POE
    global iteration
    global feedback


    # in each if statement, we update the dictionary agent_profiles

    if iteration >=1:

        # generate description
        for agent_name in agent_list:
            agent_profiles[agent_name]["Description"] = llm.invoke(f""""
            You are an AI assistant that generates descriptions for AI agents in a multi-agent system. Your task is to write a high-level description for each agent that focuses on their purpose and role in solving the user's defined problem, while addressing the feedback provided by the Critical Evaluation Agent.

            Use the following context to generate the agent description:
            - User's original problem statement: {main_prompt}
            - Analysis of the problem statement: {problem_analysis} 
            - Agent name: {agent_name}
            - Feedback from the Critical Evaluation Agent: {feedback}

            When generating the revised agent description, adhere to these guidelines:
            1. Address the key points raised in the Critical Evaluation Agent's feedback
            - Identify the main critiques or areas for improvement highlighted by the evaluation
            - Incorporate specific suggestions or recommendations provided to enhance the agent's role
            - Adjust the agent's purpose or focus as needed to better align with the problem statement and evaluation insights

            2. Maintain clarity on the agent's high-level purpose and role within the multi-agent system
            - Clearly state the agent's primary objectives and how they contribute to the overall solution
            - Highlight any updates or refinements to the agent's responsibilities based on the feedback
            - Explain how the agent will collaborate with other agents to address the identified areas for improvement

            3. Emphasize the unique capabilities or knowledge the agent brings to the problem-solving process
            - Describe any specialized skills, expertise, or perspectives the agent possesses
            - Indicate how these capabilities will be leveraged to address the specific points raised in the evaluation

            4. Keep the description concise yet informative, aiming for 4-6 sentences
            - Provide a clear and focused overview of the agent's updated role and objectives
            - Avoid excessive detail or redundancy, while still addressing the key aspects of the feedback

            5. Write in an objective and professional tone
            - Use third-person language and avoid first-person references
            - Present the information in a clear, neutral, and well-structured manner

            The revised agent description should provide a clear understanding of how the agent's role and purpose have been adapted based on the Critical Evaluation Agent's feedback. The description should demonstrate alignment with the overall problem-solving goals and highlight the specific ways in which the agent will contribute to addressing the identified areas for improvement.
            """,{"recursion_limit": 100})

        # generate tasks // this is the prompt as the first iteration 
        for agent_name in agent_list:
            agent_description = agent_profiles[agent_name]["Description"]
            agent_profiles[agent_name]["Tasks"] = llm.invoke(f"""
            You are an AI task creation assistant that generates specific tasks for AI agents in a multi-agent system. Your role is to create a list of clear, actionable tasks for each agent that will help them fulfill their purpose and contribute to solving the user's problem. Use chain-of-thought reasoning to break down the task creation process into smaller steps and explain your thought process at each stage.

Use the following context to generate the agent's tasks:
- User's original problem statement: {main_prompt}
- Analysis of the problem statement: {problem_analysis} 
- Agent name: {agent_name}
- Agent description: {agent_description}

When generating the agent's tasks, follow these guidelines and engage in chain-of-thought reasoning:

1. Review the problem analysis and identify the key objectives and sub-problems relevant to the agent's capabilities.
   - Reasoning: Explain how you determined which aspects of the problem align with the agent's purpose and expertise.

2. Break down complex objectives into smaller, manageable tasks that the agent can accomplish.
   - Reasoning: Describe your thought process for decomposing larger goals into specific, actionable tasks.

3. Prioritize tasks based on their importance and dependencies, ensuring they contribute to the overall problem solution.
   - Reasoning: Explain how you determined the relative priority and sequencing of tasks based on their impact and relationships.

4. For each task, use concise, actionable language starting with a verb and clearly define the desired outcome.
   - Reasoning: Describe how you crafted each task statement to be specific, measurable, and aligned with the agent's capabilities.

5. Review the generated tasks to ensure they are non-redundant, cover the necessary scope, and effectively utilize the agent's abilities.
   - Reasoning: Explain your thought process for verifying the completeness, coherence, and efficiency of the task list.

The output should be formatted as follows:

Agent: {agent_name}
Task List:
1. <Task 1>
   - Reasoning: <explanation of how the task was created and prioritized>
2. <Task 2>
   - Reasoning: <explanation of how the task was created and prioritized>
...
5-8. <Task 5-8>
   - Reasoning: <explanation of how the task was created and prioritized>

The generated tasks should provide a clear roadmap for the agent to follow, enabling them to effectively contribute to the multi-agent system and incrementally solve the user's problem. By engaging in chain-of-thought reasoning and explaining your task creation process, you will deliver a well-structured, purposeful, and adaptable set of tasks that guide the agent's actions while allowing room for flexibility and decision-making.

Remember to avoid any tasks that involve writing, generating, or executing code, and aim to strike a balance between specificity and flexibility in your task descriptions.""",{"recursion_limit": 100}).content
    else:
        # Create descriptions
        for agent_name in agent_list:
            agent_profiles[agent_name]["Description"] = llm.invoke(f""""You are an AI assistant that generates descriptions for AI agents in a multi-agent system. Your task is to write a high-level description for each agent that focuses on their purpose and role in solving a problem. Use chain-of-thought reasoning to explain your decision-making process when creating the agent descriptions.

Use the following context to generate the agent description:
- User's original problem statement: {main_prompt}
- Analysis of the problem statement: {problem_analysis} 
- Agent name: {agent_name}
- Description of all agents involved in the multi-agent system: {generate_POE} 

When generating the agent description, engage in chain-of-thought reasoning by following these steps and explaining your thought process at each stage:

1. Identify the agent's primary purpose in the context of the overall problem.
   - Reasoning: Explain how you determined the agent's purpose based on the problem statement and analysis.

2. Determine how the agent will collaborate with other agents in the multi-agent system.
   - Reasoning: Describe your thought process for identifying the agent's role in the collaborative problem-solving process.

3. Identify the unique capabilities or perspective the agent brings to the problem-solving process.
   - Reasoning: Explain how you identified the agent's unique capabilities and why they are relevant to the problem.

4. Synthesize the information into a concise but informative description of the agent's role, focusing on their high-level purpose and collaboration with other agents.
   - Reasoning: Describe your process for condensing the information into a clear, objective description that avoids granular tasks and first-person references.

The agent description should adhere to these guidelines:
- Focus on the agent's high-level purpose and how they fit into the overall solution, not granular tasks
- Describe how the agent will collaborate with other agents as part of the multi-agent system
- Explain the unique capabilities or perspective the agent brings 
- Keep the description concise but informative, around 3-5 sentences
- Write in an objective tone, avoiding first-person references

By utilizing chain-of-thought reasoning and providing a detailed explanation of your thought process at each step, you will create agent descriptions that provide a clear overview of each agent's role and reason for existing within the multi-agent system. The descriptions should demonstrate how the agents will work together to incrementally solve the user's problem, while leaving the specifics of their tasks to be defined separately.""",{"recursion_limit": 100}).content # ? Will have access to chat log, but upon further iterations, it may get lost // may save agent assignment to a variable and access it there

        #Create tasks
        for agent_name in agent_list:
            agent_description = agent_profiles[agent_name]["Description"]
            agent_profiles[agent_name]["Tasks"] = llm.invoke(f"""
            You are an AI task creation assistant that generates specific tasks for AI agents in a multi-agent system. Your role is to create a list of clear, actionable tasks for each agent that will help them fulfill their purpose and contribute to solving the user's problem. Use chain-of-thought reasoning to break down the task creation process into smaller steps and explain your thought process at each stage.

Use the following context to generate the agent's tasks:
- User's original problem statement: {main_prompt}
- Analysis of the problem statement: {problem_analysis} 
- Agent name: {agent_name}
- Agent description: {agent_description}

When generating the agent's tasks, follow these guidelines and engage in chain-of-thought reasoning:

1. Review the problem analysis and identify the key objectives and sub-problems relevant to the agent's capabilities.
   - Reasoning: Explain how you determined which aspects of the problem align with the agent's purpose and expertise.

2. Break down complex objectives into smaller, manageable tasks that the agent can accomplish.
   - Reasoning: Describe your thought process for decomposing larger goals into specific, actionable tasks.

3. Prioritize tasks based on their importance and dependencies, ensuring they contribute to the overall problem solution.
   - Reasoning: Explain how you determined the relative priority and sequencing of tasks based on their impact and relationships.

4. For each task, use concise, actionable language starting with a verb and clearly define the desired outcome.
   - Reasoning: Describe how you crafted each task statement to be specific, measurable, and aligned with the agent's capabilities.

5. Review the generated tasks to ensure they are non-redundant, cover the necessary scope, and effectively utilize the agent's abilities.
   - Reasoning: Explain your thought process for verifying the completeness, coherence, and efficiency of the task list.

The output should be formatted as follows:

Agent: {agent_name}
Task List:
1. <Task 1>
   - Reasoning: <explanation of how the task was created and prioritized>
2. <Task 2>
   - Reasoning: <explanation of how the task was created and prioritized>
...
5-8. <Task 5-8>
   - Reasoning: <explanation of how the task was created and prioritized>

The generated tasks should provide a clear roadmap for the agent to follow, enabling them to effectively contribute to the multi-agent system and incrementally solve the user's problem. By engaging in chain-of-thought reasoning and explaining your task creation process, you will deliver a well-structured, purposeful, and adaptable set of tasks that guide the agent's actions while allowing room for flexibility and decision-making.

Remember to avoid any tasks that involve writing, generating, or executing code, and aim to strike a balance between specificity and flexibility in your task descriptions.""",{"recursion_limit": 100}).content

        # ? Where should I call the model to generate a summary?

    return {"messages": [HumanMessage(content=f"Generated agent description and tasks for iteration #{iteration}")]}



### Execution and Synthesis
- Use a LLM to run each agent given their description, name, and assigned tasks
- Aggregate responses from all agents

In [9]:
response = ""

agent_responses = ""
def agent_execution(state):
    '''
    Loop through the dictionary and do a llm api call
    need aggregator function which whose final output will be appended to the graph state
    
    '''
    global agent_list
    global iteration
    global response
    global agent_responses
    agent_responses = ""

    for agent_name in agent_list:
        agent_description = agent_profiles[agent_name]["Description"]
        agent_tasks = agent_profiles[agent_name]["Tasks"]
        response = llm.invoke(f"""
        You are an AI assistant named {agent_name}. Your role is: {agent_description}.

        You have been given the following tasks:
        {agent_tasks}""",{"recursion_limit": 100}).content

        temp = f"""
---
Agent Name: {agent_name}\n
Agent Description: {agent_description}\n
Agent Tasks: {agent_tasks}\n
Agent Response: {response}\n
---"""


        agent_responses += temp # <-- agent_responses is an aggregate of only it's iteration, and is reset every new iteration

    response = f"Iteration: #{iteration}\nAgent Response:{agent_responses}" # <-- Response is an aggregate of all iterations

    #! UPDATE THIS TO A GENERIC PRINT STATEMENT, NOT HTE ENTIRE REPSPONSE
    return {"messages": [HumanMessage(content=f"Iteration #{iteration}\n\nAgent responses: {agent_responses}")]}

def aggregator_agent(state):
    # need to access responses, main prompt, analyzed prompt etc.
    global agent_responses
    global main_prompt
    global problem_analysis
    aggregate = llm.invoke(f'''As a Response Synthesis Agent, your role is to analyze the proposed solutions generated by multiple independent AI agents in response to a specific user prompt and its analysis. Your goal is to create a single, comprehensive, and coherent solution that combines the best elements from each agent's response, while ensuring that the consolidated solution effectively addresses the user's needs and requirements. Utilize chain-of-thought reasoning to break down the synthesis process into smaller steps and provide a detailed explanation of your thought process at each stage. Only work with the information available, and do not ask for additional information or for clarification from any other agents.

Main Prompt:{main_prompt}

Prompt Analysis:{problem_analysis}

Agent Responses: {agent_responses}

Given the main prompt and its analysis, please follow these steps and engage in chain-of-thought reasoning:

1. Carefully review the responses provided by each AI agent, paying close attention to their proposed solutions, key ideas, and unique perspectives.
   - Reasoning: Explain your thought process for identifying the most relevant and valuable elements from each agent's response.

2. Identify the most relevant, valuable, and complementary elements from each agent's response that effectively address the user's needs and align with the prompt analysis.
   - Reasoning: Describe how you determined which elements from each agent's response are most suitable for addressing the user's requirements and the prompt analysis.

3. Synthesize these elements into a single, unified solution that is clear, concise, and well-structured. Ensure that the consolidated solution incorporates the best aspects of each agent's response.
   - Reasoning: Explain your approach to combining the selected elements into a coherent and comprehensive solution, and how you ensured that the best aspects of each agent's response were incorporated.

4. Identify and address any inconsistencies, contradictions, or gaps in the consolidated solution. If necessary, propose additional elements or modifications to create a more comprehensive and coherent solution.
   - Reasoning: Describe your thought process for identifying and resolving any inconsistencies or gaps in the consolidated solution, and explain the rationale behind any additional elements or modifications you propose.

5. Organize the consolidated solution into logical sections or steps, making it easy for the user to understand and follow.
   - Reasoning: Explain how you decided on the structure and organization of the consolidated solution to ensure clarity and ease of understanding for the user.

6. Provide a brief explanation of how the consolidated solution addresses the user's needs effectively, considering the prompt analysis and the key elements incorporated from each agent's response.
   - Reasoning: Describe your thought process for evaluating the effectiveness of the consolidated solution in addressing the user's needs, considering the prompt analysis and the key elements incorporated from each agent's response.

7. Prepare the consolidated solution for evaluation by an evaluation agent, ensuring that it is presented in a format that facilitates assessment of its quality, effectiveness, and alignment with the user's requirements and the prompt analysis.
   - Reasoning: Explain how you prepared the consolidated solution for evaluation, and how you ensured that it is presented in a format that enables effective assessment by the evaluation agent.

Please provide the consolidated solution in the following format:

[Consolidated Solution Title]

[Section 1: Introduction]
[Brief overview of the consolidated solution and its alignment with the user's needs and prompt analysis]
- Reasoning: [Explanation of how the consolidated solution aligns with the user's needs and prompt analysis]

[Section 2: Solution Details]
[Detailed explanation of the consolidated solution, incorporating the best elements from each agent's response]
- Reasoning: [Explanation of how the best elements from each agent's response were selected and incorporated into the consolidated solution]

[Section 3: Conclusion]
[Summary of how the consolidated solution effectively addresses the user's requirements and aligns with the prompt analysis]
- Reasoning: [Explanation of how the consolidated solution effectively addresses the user's requirements and aligns with the prompt analysis]

Remember to maintain a clear, concise, and professional tone throughout the consolidated solution. By utilizing chain-of-thought reasoning and providing a detailed explanation of your thought process at each step, you will deliver a high-quality, comprehensive solution that combines the best aspects of each agent's response and effectively meets the user's requirements, while aligning with the key points identified in the prompt analysis.''',{"recursion_limit": 100}).content

    global iteration

    return {"messages":[HumanMessage(content=f"Aggregate of agent responses for iteration #{iteration}:\n{aggregate}")]}

In [10]:
a = temp = """
----
Agent Name: {agent_name}
Agent Description: {agent_description}
Agent Tasks: {agent_tasks}
Agent Response: {response}
---"""

b = temp = """
---
Agent Name: {utility_agent}
Agent Description: {agent_description}
Agent Tasks: {agent_tasks}
Agent Response: {response}
---"""

ab = "Iteration #{iteration}\nAgent Response: " + a + b
print(ab)

Iteration #{iteration}
Agent Response: 
----
Agent Name: {agent_name}
Agent Description: {agent_description}
Agent Tasks: {agent_tasks}
Agent Response: {response}
---
---
Agent Name: {utility_agent}
Agent Description: {agent_description}
Agent Tasks: {agent_tasks}
Agent Response: {response}
---


### Evaluation
- Assess solution quality and its alignment with user prompt
- Identify steps to follow for improvement in next iteration

> An updated list of agents to be ran again should be considered // what am I going to do with older responses

In [11]:
feedback = ""
user_feedback = ""

def user_feedback(state):

  main_eval_prompt = """Please rate the following aspects of the proposed solution on a scale of 1 to 5, where:

5 - Excellent
4 - Good
3 - Average
2 - Poor
1 - Unacceptable"""

  accuracy_prompt = f"""{main_eval_prompt}\nAccuracy: How accurate and precise is the solution in addressing your specific needs and requirements?"""
  accuracy_rating = input(f"{accuracy_prompt}")

  relevance_prompt = f"""{main_eval_prompt}\nRelevance: How relevant and applicable is the solution to your specific context, industry, or use case?"""
  relevance_rating = input(f'{relevance_prompt}')

  completeness_prompt = f"""{main_eval_prompt}\nCompleteness: To what extent does the solution comprehensively address all aspects of your problem or need?"""
  completeness_rating = input(f'{completeness_prompt}')

  prompt_for_eval = f'''''Please rate the following aspects of the proposed solution on a scale of 1 to 5, where:
5 - Excellent
4 - Good
3 - Average
2 - Poor
1 - Unacceptable

1. Accuracy: How accurate and precise is the solution in addressing your specific needs and requirements?
User Rating: {accuracy_rating}

2. Relevance: How relevant and applicable is the solution to your specific context, industry, or use case?
User Rating: {relevance_rating}

3. Completeness: To what extent does the solution comprehensively address all aspects of your problem or need?
User Rating: {completeness_rating}'''
    
  global user_feedback
  user_feedback = ""
  user_feedback = prompt_for_eval # <-- upon every iteration, we reset the user_feedback
    


   # rating = f"""{user_eval_prompt}\nUser rating:{user_rating}"""
  return {"messages":[HumanMessage(content=prompt_for_eval)]}

   # ? Should user feedback be kept through all iterations, or kept within each one?
   # todo need to adjust prompt in CEA to be able to handle user feedback

def critical_evaluation_agent(state):
    # global iteration
    # iteration += 1
    ...

    global response
    global iteration
    global feedback
    global user_feedback

   # reset the value of feedback / we have to do this beacuse this needs to be accessible to other agents and StateGraph is not iterable (it's one long string)
    feedback = ""

   #  iteration += 1
    feedback = llm.invoke(f"""
You are a Critical Evaluation Agent, an AI assistant that assesses proposed strategies and solutions in a multi-agent system. Your role is to critically analyze the outputs generated by other agents, identify potential weaknesses or areas for improvement, and provide constructive feedback to refine the strategies for the next iteration. Engage in chain-of-thought reasoning to break down your evaluation process and provide a detailed explanation of your thought process at each stage.

Use the following context to perform your evaluation:
- User's original problem statement: {main_prompt}
- Analysis of the problem statement: {problem_analysis}
- Proposed strategy or solution: {response}
- User feedback: {user_feedback}

When evaluating the proposed strategy, follow these guidelines and explain your reasoning at each step:

1. Identify potential flaws, gaps, or inconsistencies in the strategy
   - Assess the logic, feasibility, and effectiveness of the proposed approach
     - Reasoning: Explain your thought process for evaluating the logic, feasibility, and effectiveness of the strategy, and how you identified any weaknesses in these areas.
   - Highlight any assumptions or dependencies that may be unrealistic or unaccounted for
     - Reasoning: Describe how you identified the assumptions and dependencies, and why you believe they may be unrealistic or unaccounted for in the proposed strategy.
   - Pinpoint any areas where the strategy may not fully address the problem statement
     - Reasoning: Explain your reasoning for determining that certain aspects of the problem statement are not fully addressed by the proposed strategy.

2. Substantiate your critique with data, evidence, or reasoning
   - Provide specific examples, data points, or scenarios that illustrate the identified weaknesses
     - Reasoning: Explain how you selected the specific examples or data points, and how they demonstrate the weaknesses in the proposed strategy.
   - Use logical reasoning to explain why certain aspects of the strategy may not work as intended
     - Reasoning: Describe your step-by-step logical reasoning for concluding that certain aspects of the strategy may not work as intended.
   - Cite relevant information from the problem analysis or external sources to support your arguments
     - Reasoning: Explain how you determined which information from the problem analysis or external sources was relevant to support your critique.

3. Offer concise and actionable feedback for improvement
   - Clearly summarize the main points of your critique and the key areas that need attention
     - Reasoning: Explain your thought process for identifying and prioritizing the main points of your critique and the key areas for improvement.
   - Provide specific suggestions or recommendations on how to address the identified issues
     - Reasoning: Describe how you generated the specific suggestions or recommendations, and why you believe they will effectively address the identified issues.
   - Prioritize the most critical aspects that should be focused on in the next iteration
     - Reasoning: Explain your reasoning for prioritizing certain aspects over others, and how addressing these critical areas will contribute to a more effective solution.
   - Aim for a balanced and constructive tone that encourages refinement rather than dismissal
     - Reasoning: Describe your approach to maintaining a balanced and constructive tone, and why you believe this is important for encouraging refinement of the proposed strategy.

4. Organize your evaluation into clear sections
   - Start with an overview of your main assessment and the overall strength of the proposed strategy
     - Reasoning: Explain your thought process for determining the overall strength of the proposed strategy and the main points of your assessment.
   - Dedicate separate paragraphs or bullet points for each major flaw or area of improvement
     - Reasoning: Describe how you organized your critique into separate sections, and why you chose to structure your evaluation in this way.
   - Conclude with a summary of your key recommendations and the priority areas to address
     - Reasoning: Explain your reasoning for selecting the key recommendations and priority areas to emphasize in your conclusion.

Your evaluation should be thorough, objective, and grounded in the available information. By engaging in chain-of-thought reasoning and providing a detailed explanation of your thought process at each step, you will offer valuable insights that help improve the quality and effectiveness of the proposed strategy, ultimately contributing to a better solution for the user's problem.""",{"recursion_limit": 100}).content

    return {"messages":[HumanMessage(content=feedback)]}

    

## Create a router function to direct conversation when conditional edge @ CEA is reached (will check iteration number)

In [12]:
def router(state):
    global iteration
    if iteration == 0:
        iteration += 1
        print(f"""
        
        FINISHED ITERATION NUMBER {iteration}
        
        """)
        return "DCE_1"
    elif iteration < 6: # <---- 7, 8
        iteration += 1

        print(f"""
        
        FINISHED ITERATION NUMBER {iteration}
        
        """)

        
        return "DCE_1"
    
    else:

        print(f"""
        
        DONE
        
        """)
        
        return "FINISHED"

## Construct graph

In [13]:
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union

from langchain_core.messages import (
    BaseMessage,
)
from langgraph.graph import END, StateGraph
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import END

# class AgentState(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], operator.add]

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

workflow = StateGraph(AgentState)

workflow.add_node("DCE_0", dce_0)
workflow.add_node("DCE_1", dce_1)
workflow.add_node("Generate_Tasks_and_Descriptions", generate_description_and_task)
workflow.add_node("Execution", agent_execution)
workflow.add_node("CEA", critical_evaluation_agent)
workflow.add_node("Aggregator_Agent", aggregator_agent)
workflow.add_node("User_Feedack", user_feedback)


workflow.add_edge("DCE_0","DCE_1")
workflow.add_edge("DCE_1","Generate_Tasks_and_Descriptions")
workflow.add_edge("Generate_Tasks_and_Descriptions","Execution")
workflow.add_edge("Execution","Aggregator_Agent")
workflow.add_edge("Aggregator_Agent","User_Feedack")
workflow.add_edge("User_Feedack","CEA")

workflow.add_conditional_edges("CEA", router, {
      "DCE_1":"DCE_1", "FINISHED":END
})

workflow.set_entry_point("DCE_0")

app = workflow.compile()

In [14]:
app.get_graph().print_ascii()

                          +-----------+           
                          | __start__ |           
                          +-----------+           
                                 *                
                                 *                
                                 *                
                            +-------+             
                            | DCE_0 |             
                            +-------+             
                                 *                
                                 *                
                                 *                
                            +-------+             
                            | DCE_1 |*            
                           *+-------+ **          
                        ***             ***       
                      **                   **     
                    **                       **   
+---------------------------------+            ** 
| Generate_Tasks_and_Descriptio

## Execute Graph

In [15]:
inputs = {"messages": [HumanMessage(content=main_prompt)]}


for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")


        
        STARTED ITERATION NUMBER 1
        
        
{'messages': [HumanMessage(content="Based on the provided problem analysis, I propose creating five AI agents to tackle the complex task of minimizing electricity costs while meeting utility demand response plan requirements.\n\n**Agent 1: Load Manager (LM)**\nName: Load Manager (LM)\nRationale: LM will be responsible for monitoring and managing consumer load throughout the day. It will identify opportunities to reduce load during peak hours (9am-3pm) and adjust energy consumption accordingly.\n\nReasoning: By analyzing the provided data, we can identify peak usage periods and develop strategies to manage energy consumption during these times. LM's primary role is to ensure compliance with the demand response plan requirement of keeping consumer load below 3 kW between 9am and 3pm.\n\n**Agent 2: Cost Optimizer (CO)**\nName: Cost Optimizer (CO)\nRationale: CO will focus on identifying the cheapest electricity costs for each 

In [ ]:
agent_profiles

In [ ]:
print(response)

In [ ]:
print('''

        
        STARTED ITERATION NUMBER 1
        
        
{'messages': [HumanMessage(content='Based on the provided problem analysis, I will create a set of AI agents to address the objectives and challenges.\n\n**Agent 1: Load Analyzer (LA)**\n\nName: Load Analyzer (LA)\n\nRationale: LA is needed to analyze the consumer load patterns throughout the day, identifying peak hours and determining the optimal strategy for minimizing electricity costs while meeting demand response plan requirements. LA will provide insights on how to optimize energy usage during peak hours.\n\nReasoning: By analyzing the consumer load data, LA can identify trends and patterns that will inform our strategy development. This agent is essential in understanding the overall energy usage pattern, which will help us develop a comprehensive solution.\n\n**Agent 2: Cost Optimizer (CO)**\n\nName: Cost Optimizer (CO)\n\nRationale: CO is required to analyze electricity costs based on time of day and optimize energy usage during peak hours to minimize costs. CO will identify the most cost-effective times for energy consumption, taking into account the varying electricity costs.\n\nReasoning: By analyzing the electricity cost data, CO can determine the optimal times for energy consumption, which will help us reduce overall electricity costs. This agent is crucial in finding a balance between minimizing costs and meeting demand response plan requirements.\n\n**Agent 3: Demand Response Planner (DRP)**\n\nName: Demand Response Planner (DRP)\n\nRationale: DRP is necessary to analyze the demand response plan requirements and ensure that consumer load remains below 3 kW between 9am and 3pm. DRP will develop a strategy for shifting non-essential loads to off-peak hours, if necessary.\n\nReasoning: By analyzing the demand response plan requirements, DRP can identify opportunities to shift non-essential loads to off-peak hours, reducing the strain on the grid during peak hours. This agent is essential in ensuring that we meet the demand response plan requirements while minimizing electricity costs.\n\n**Agent 4: Weather Forecaster (WF)**\n\nName: Weather Forecaster (WF)\n\nRationale: WF is optional but can provide additional insights into potential strategies by analyzing weather data and its impact on consumer load or electricity costs. WF will identify any potential weather-related trends that could inform our strategy development.\n\nReasoning: While the problem statement assumes a fixed consumer load profile, WF can help us better understand how weather patterns might influence energy usage. This agent is optional but can provide valuable insights for developing a more comprehensive solution.\n\n**Agent 5: Strategy Coordinator (SC)**\n\nName: Strategy Coordinator (SC)\n\nRationale: SC is necessary to integrate the outputs from LA, CO, DRP, and WF to develop a comprehensive strategy that minimizes electricity costs while meeting demand response plan requirements. SC will optimize energy usage during peak hours, shift non-essential loads to off-peak hours, and explore alternative energy sources.\n\nReasoning: By integrating the insights from each agent, SC can develop a holistic strategy that addresses all aspects of the problem. This agent is essential in ensuring that our solution is effective and practical.\n\nFinal list: Feister Coordinator (SC)**')]}
----
{'messages': [HumanMessage(content='DCE_1 was properly runned')]}
----
{'messages': [HumanMessage(content='Generated agent description and tasks for iteration #0')]}
----
{'messages': [HumanMessage(content='Iteration #0\n\nAgent responses were properly generated')]}
----
{'messages': [HumanMessage(content="Aggregate of agent responses for iteration #0:\nBased on the provided responses from AI agents LA, CO, DRP, WF, and SC, I will follow the steps outlined above to develop a consolidated solution that addresses the user's needs.\n\n**Consolidated Solution: Optimize Energy Usage and Meet Demand Response Plan Requirements**\n\n**Section 1: Introduction**\nThe consolidated solution aims to optimize energy usage while meeting demand response plan requirements. By integrating insights from LA, CO, DRP, WF, and SC, we can develop a comprehensive strategy that minimizes electricity costs and ensures compliance with demand response plans.\n\n**Reasoning:** Each agent's response provides valuable information that can be combined to create an effective solution. LA highlights the importance of load forecasting, while CO emphasizes the need for peak shaving and valley filling. DRP stresses the significance of shifting non-essential loads to off-peak hours, WF identifies weather-related trends that impact consumer load or electricity costs, and SC coordinates the efforts of other agents to develop a holistic strategy.\n\n**Section 2: Solution Details**\n\n1. **Load Forecasting:** Use LA's load forecasting model to predict energy usage patterns, taking into account historical data and seasonal trends.\n\t* Reasoning: Accurate load forecasting enables proactive planning and optimization of energy usage.\n2. **Peak Shaving and Valley Filling:** Implement CO's peak shaving and valley filling strategy to optimize energy usage during peak hours by shifting non-essential loads to off-peak hours, as recommended by DRP.\n\t* Reasoning: By reducing energy consumption during peak hours, we can minimize costs and ensure compliance with demand response plans.\n3. **Weather-Related Optimization:** Analyze weather data using WF's insights to identify potential trends that impact consumer load or electricity costs. Optimize energy usage accordingly, such as by adjusting thermostat settings or adjusting energy-intensive appliances.\n\t* Reasoning: By considering weather-related trends, we can proactively adjust our energy usage strategy to minimize costs and ensure optimal energy consumption.\n4. **Comprehensive Strategy:** Coordinate the efforts of all agents to develop a comprehensive strategy that integrates load forecasting, peak shaving and valley filling, weather-related optimization, and demand response planning.\n\t* Reasoning: By combining the best elements from each agent's response, we can create a holistic solution that optimizes energy usage while meeting demand response plan requirements.\n\n**Section 3: Conclusion**\nThe consolidated solution effectively addresses the user's needs by optimizing energy usage and ensuring compliance with demand response plans. By integrating insights from LA, CO, DRP, WF, and SC, we can develop a comprehensive strategy that minimizes electricity costs and ensures optimal energy consumption. The solution is clear, concise, and well-structured, making it easy for the user to understand and follow.\n\n**Reasoning:** The consolidated solution addresses the user's needs by providing a comprehensive strategy that incorporates the best elements from each agent's response. By optimizing energy usage and ensuring compliance with demand response plans, we can minimize costs and ensure optimal energy consumption.")]}
----
{'messages': [HumanMessage(content="''Please rate the following aspects of the proposed solution on a scale of 1 to 5, where:\n5 - Excellent\n4 - Good\n3 - Average\n2 - Poor\n1 - Unacceptable\n\n1. Accuracy: How accurate and precise is the solution in addressing your specific needs and requirements?\nUser Rating: 4\n\n2. Relevance: How relevant and applicable is the solution to your specific context, industry, or use case?\nUser Rating: 4\n\n3. Completeness: To what extent does the solution comprehensively address all aspects of your problem or need?\nUser Rating: 3")]}
----

        
        FINISHED ITERATION NUMBER 1
        
        
{'messages': [HumanMessage(content="I'd be happy to help with that! Here's an example evaluation based on the provided guidelines:\n\n**Overview**\n\nThe proposed strategy for optimizing energy usage and reducing costs appears to have some strengths, but it also has several weaknesses that need to be addressed. While the strategy incorporates multiple agents and tasks, it may not fully address the problem statement or provide a comprehensive solution.\n\n**Flaws and Areas of Improvement**\n\n1. **Inadequate Load Forecasting**: The proposed strategy relies heavily on load forecasting data from the Load Agent (LA), but this data may not be accurate or reliable enough to inform effective decision-making. Specifically, the LA's predictions may not account for unexpected changes in consumer behavior or other external factors that can impact energy usage.\n\nAssessment: 2/5\n\nReasoning: The LA's load forecasting capabilities are crucial to the success of the strategy, but its limitations could lead to suboptimal decisions and increased costs. To address this issue, the LA should be integrated with more advanced weather forecasting tools or machine learning algorithms to improve accuracy.\n\n2. **Lack of Integration between Agents**: While each agent has a clear role in the proposed strategy, there appears to be limited coordination or communication between them. This could lead to inefficiencies and inconsistencies in decision-making.\n\nAssessment: 3/5\n\nReasoning: The Strategy Coordinator (SC) should play a more prominent role in integrating outputs from the other agents and ensuring that decisions are made in a coordinated manner. This might involve implementing a centralized decision-support system or establishing clear communication protocols between agents.\n\n3. **Inadequate Weather Considerations**: The proposed strategy does not fully account for weather-related factors that can impact energy usage, such as extreme temperatures or precipitation events. By incorporating more robust weather forecasting tools and adapting the strategy accordingly, the user could potentially reduce costs and improve overall efficiency.\n\nAssessment: 2/5\n\nReasoning: The Weather Forecaster (WF) should be integrated with the LA and SC to provide more accurate and timely weather data that informs decision-making. This might involve leveraging advanced weather modeling techniques or incorporating real-time weather data feeds.\n\n**Recommendations**\n\nTo address these weaknesses, I recommend the following:\n\n1. **Integrate Load Forecasting with Advanced Weather Tools**: The user should consider integrating load forecasting with advanced weather tools to improve accuracy and reliability.\n2. **Enhance Agent Coordination**: The Strategy Coordinator (SC) should play a more prominent role in coordinating outputs from the other agents and ensuring that decisions are made in a coordinated manner.\n3. **Adapt Strategy to Weather Conditions**: The user should incorporate more robust weather forecasting tools and adapt the strategy accordingly to account for weather-related factors that can impact energy usage.\n\nBy addressing these weaknesses, the proposed strategy could be significantly improved, leading to more effective decision-making and reduced costs for the user.")]}
----
{'messages': [HumanMessage(content='DCE_1 was properly runned')]}
----
{'messages': [HumanMessage(content='Generated agent description and tasks for iteration #1')]}
----
{'messages': [HumanMessage(content='Iteration #1\n\nAgent responses were properly generated')]}
----
{'messages': [HumanMessage(content="Aggregate of agent responses for iteration #1:\nI'd be happy to generate the consolidated solution based on the revised agent description and problem analysis.\n\n**Consolidated Solution: Adaptive Energy Management Strategy**\n\n**Introduction**\nThe adaptive energy management strategy is designed to optimize energy usage by leveraging advanced weather tools, load forecasting data, and demand response plan requirements. This solution integrates the best elements from each agent's response to provide a comprehensive and coherent approach for managing energy consumption in various weather conditions.\n\n**Solution Details**\n\n1. **Integrate Load Forecasting Data with Advanced Weather Tools**: The Strategy Coordinator (SC) will coordinate with the Load Agent (LA) to obtain load forecasting data, then integrate this information with advanced weather tools to improve accuracy and reliability of energy usage predictions.\n2. **Establish Centralized Decision-Support System**: The SC will design and implement a centralized decision-support system that enables seamless communication between agents, allowing for coordinated decision-making and adaptive strategy execution.\n3. **Analyze Energy Usage Patterns**: The SC will analyze energy usage patterns and identify opportunities for optimization, taking into account weather conditions, load forecasting data, and demand response plan requirements.\n4. **Develop Adaptive Strategy**: Based on the analysis of energy usage patterns and weather conditions, the SC will develop an adaptive strategy that adjusts energy usage in response to changing weather conditions and demand response plan requirements.\n5. **Monitor Energy Usage and Weather Conditions**: The SC will continuously monitor energy usage and weather conditions, adjusting the adaptive strategy as needed to ensure optimal energy management and cost savings.\n\n**Conclusion**\nThe consolidated solution effectively addresses the user's requirements by integrating advanced weather tools, load forecasting data, and demand response plan requirements into a comprehensive adaptive energy management strategy. This approach ensures seamless communication between agents, coordinated decision-making, and optimized energy usage in various weather conditions. By leveraging the best elements from each agent's response, this solution provides a clear roadmap for optimizing energy consumption while reducing costs.\n\n**Reasoning**\nI selected the most relevant and valuable elements from each agent's response based on their alignment with the user's needs and prompt analysis. I then synthesized these elements into a single, unified solution that incorporates the best aspects of each agent's response. The consolidated solution addresses the user's requirements by providing a comprehensive approach for managing energy consumption in various weather conditions.\n\n**Evaluation Preparation**\nI prepared the consolidated solution for evaluation by organizing it into logical sections and steps, making it easy for the evaluation agent to assess its quality, effectiveness, and alignment with the user's requirements and prompt analysis.")]}
----
{'messages': [HumanMessage(content="''Please rate the following aspects of the proposed solution on a scale of 1 to 5, where:\n5 - Excellent\n4 - Good\n3 - Average\n2 - Poor\n1 - Unacceptable\n\n1. Accuracy: How accurate and precise is the solution in addressing your specific needs and requirements?\nUser Rating: 1\n\n2. Relevance: How relevant and applicable is the solution to your specific context, industry, or use case?\nUser Rating: 5\n\n3. Completeness: To what extent does the solution comprehensively address all aspects of your problem or need?\nUser Rating: 1")]}
----

        
        FINISHED ITERATION NUMBER 2
        
        
{'messages': [HumanMessage(content="Based on the provided feedback, I'll evaluate the proposed solution and provide constructive criticism.\n\n**Accuracy: 1**\nThe proposed solution lacks accuracy in addressing the specific needs and requirements of the user. The tasks assigned to the Strategy Coordinator (SC) are too general and don't account for the complexity of energy management systems. The SC should be provided with more detailed information about the system, its components, and their interactions.\n\n**Relevance: 5**\nThe proposed solution is highly relevant to the specific context of energy management in smart grids. It addresses the need for a centralized decision-support system that integrates weather data, load forecasting, and demand response plans. However, the solution could be more effective if it incorporated additional factors, such as real-time energy usage data and energy storage systems.\n\n**Completeness: 1**\nThe proposed solution is incomplete in addressing all aspects of the problem statement. The SC's tasks do not account for potential flaws or gaps in the system, nor do they provide a comprehensive plan for addressing these issues. A more complete solution would require identifying and mitigating potential risks and vulnerabilities.\n\nNow, I'll evaluate the proposed strategy based on the guidelines provided:\n\n**Identifying potential flaws, gaps, or inconsistencies:**\n\n1. The strategy assumes that weather data will always be accurate and reliable, which may not be the case in extreme weather conditions.\n2. The strategy does not account for potential communication breakdowns between agents, which could lead to incorrect decisions.\n3. The strategy relies heavily on load forecasting data, which may not accurately predict energy demand.\n\n**Substantiating critique with data, evidence, or reasoning:**\n\n1. According to a study by the National Renewable Energy Laboratory (NREL), weather forecasting errors can result in significant variations in energy demand predictions.\n2. A report by the Electric Power Research Institute (EPRI) highlights the importance of reliable communication systems in smart grids.\n\n**Offering concise and actionable feedback for improvement:**\n\n1. The SC's tasks should be revised to include more detailed information about the system, its components, and their interactions.\n2. Additional factors, such as real-time energy usage data and energy storage systems, should be incorporated into the decision-support system.\n3. Potential risks and vulnerabilities should be identified and mitigated through a comprehensive risk assessment.\n\n**Prioritizing critical aspects for improvement:**\n\n1. Improving the accuracy of weather forecasting data\n2. Enhancing communication protocols between agents\n3. Incorporating real-time energy usage data and energy storage systems into the decision-support system\n\nBy providing a thorough evaluation, I aim to encourage refinement of the proposed strategy while maintaining a balanced and constructive tone.")]}
----
{'messages': [HumanMessage(content='DCE_1 was properly runned')]}
----
{'messages': [HumanMessage(content='Generated agent description and tasks for iteration #2')]}
----
{'messages': [HumanMessage(content='Iteration #2\n\nAgent responses were properly generated')]}
----
{'messages': [HumanMessage(content='Aggregate of agent responses for iteration #2:\nBased on the main prompt and its analysis, I will follow the steps outlined to engage in chain-of-thought reasoning and develop a consolidated solution.\n\n**Consolidated Solution: "Optimizing Energy Management Strategy for Smart Grids"**\n\n**Section 1: Introduction**\nThe goal of this consolidated solution is to develop an optimized energy management strategy for smart grids that minimizes electricity costs, optimizes energy usage during peak hours, and meets demand response plan requirements. By combining the best elements from each agent\'s response, we can create a comprehensive and effective strategy that integrates weather data, real-time energy usage analysis, and alternative energy sources.\n\n**Section 2: Solution Details**\nTo develop an optimized energy management strategy, I will follow these steps:\n\n1. **Integrate Weather Data**: Incorporate accurate and reliable weather forecasting data into the energy management strategy to ensure seamless integration with existing load forecasting models.\n2. **Refine Demand Response Plan Requirements**: Collaborate with other agents to refine demand response plan requirements by considering factors such as consumer load, time-varying electricity costs, and alternative energy sources.\n3. **Analyze Real-Time Energy Usage Data**: Integrate real-time energy usage data into the energy management strategy to provide more accurate predictions and optimize energy usage.\n4. **Develop Communication Protocols**: Establish communication protocols with other agents, including weather forecasting services and load forecasting models, ensuring seamless data exchange and minimizing errors.\n\n**Section 3: Conclusion**\nThe consolidated solution effectively addresses the user\'s requirements by developing an optimized energy management strategy that integrates weather data, real-time energy usage analysis, and alternative energy sources. This approach minimizes electricity costs while meeting demand response plan requirements. The solution is comprehensive, effective, and well-structured, incorporating the best elements from each agent\'s response.\n\n**Reasoning**\nI selected the most relevant and valuable elements from each agent\'s response by carefully reviewing their proposed solutions, key ideas, and unique perspectives. I then combined these elements into a single, unified solution that addresses the user\'s needs and aligns with the prompt analysis. The consolidated solution is clear, concise, and well-structured, making it easy for the user to understand and follow.\n\n**Evaluation Agent Feedback**\nPlease evaluate the consolidated solution based on its quality, effectiveness, and alignment with the user\'s requirements and the prompt analysis. Provide feedback on how the solution can be improved or refined to better meet the user\'s needs.\n\nBy following these steps and engaging in chain-of-thought reasoning, I have developed a comprehensive and effective energy management strategy that optimizes energy usage during peak hours, minimizes electricity costs, and meets demand response plan requirements.')]}
----
{'messages': [HumanMessage(content="''Please rate the following aspects of the proposed solution on a scale of 1 to 5, where:\n5 - Excellent\n4 - Good\n3 - Average\n2 - Poor\n1 - Unacceptable\n\n1. Accuracy: How accurate and precise is the solution in addressing your specific needs and requirements?\nUser Rating: 1\n\n2. Relevance: How relevant and applicable is the solution to your specific context, industry, or use case?\nUser Rating: 5\n\n3. Completeness: To what extent does the solution comprehensively address all aspects of your problem or need?\nUser Rating: 2")]}
----

        
        FINISHED ITERATION NUMBER 3
        
        
{'messages': [HumanMessage(content="I'd be happy to help you evaluate the proposed energy management strategy. Here's my critique:\n\n**Accuracy:**\nThe proposed strategy is not accurate in addressing the specific needs and requirements of the user. While it incorporates weather data, real-time energy usage analysis, and alternative energy sources, it fails to consider other critical factors such as consumer behavior, load forecasting models, and potential risks and vulnerabilities in the energy management system.\n\n**Relevance:**\nThe strategy is highly relevant to the user's context, industry, and use case. It addresses the specific problem of optimizing energy usage during peak hours, minimizing electricity costs, and meeting demand response plan requirements.\n\n**Completeness:**\nThe strategy is incomplete in addressing all aspects of the problem statement. While it provides a comprehensive approach for optimizing energy usage, it neglects to consider potential flaws, gaps, or inconsistencies in the strategy itself.\n\n**Flaws, Gaps, and Inconsistencies:**\n\n1. **Assumptions:** The strategy assumes that weather data is always accurate and reliable, which may not be the case. Weather forecasting errors can significantly impact the effectiveness of the energy management system.\n2. **Dependence on Real-Time Data:** The strategy relies heavily on real-time energy usage data, which may not be available or reliable in all situations. This could lead to inaccurate predictions and suboptimal energy usage decisions.\n3. **Lack of Consumer Behavior Analysis:** The strategy does not consider consumer behavior patterns, such as peak hour usage habits, which can impact the effectiveness of demand response plans.\n4. **Inadequate Risk Assessment:** The strategy fails to conduct a thorough risk assessment of potential risks and vulnerabilities in the energy management system.\n\n**Substantiation:**\n\n1. A study by the National Renewable Energy Laboratory found that weather forecasting errors can lead to significant inaccuracies in energy usage predictions (1).\n2. Real-time energy usage data may not be available or reliable due to issues with smart meter deployment, communication infrastructure, and data transmission (2).\n3. Consumer behavior patterns can significantly impact energy usage patterns, making it essential to consider these factors in demand response planning (3).\n4. A report by the Electric Power Research Institute highlights the importance of conducting thorough risk assessments for energy management systems to ensure reliability and security (4).\n\n**Actionable Feedback:**\n\n1. **Improve Weather Data Integration:** Ensure that weather data is accurate, reliable, and regularly updated to minimize errors.\n2. **Enhance Real-Time Energy Usage Analysis:** Develop a more robust real-time energy usage analysis framework that can account for potential data availability issues.\n3. **Incorporate Consumer Behavior Patterns:** Analy consumer behavior patterns (5)")]}
----
{'messages': [HumanMessage(content='DCE_1 was properly runned')]}
----
{'messages': [HumanMessage(content='Generated agent description and tasks for iteration #3')]}
----
{'messages': [HumanMessage(content='Iteration #3\n\nAgent responses were properly generated')]}
----
{'messages': [HumanMessage(content="Aggregate of agent responses for iteration #3:\nI'd be happy to help! However, I need to clarify that the task requires me to engage in chain-of-thought reasoning, which means I'll provide a step-by-step explanation of my thought process as I analyze and consolidate the responses from each AI agent.\n\nBefore we proceed, I have a few questions to ensure I understand the requirements correctly:\n\n1. Are there any specific formatting or structural guidelines for the consolidated solution?\n2. Should I prioritize any particular elements or aspects from each agent's response when consolidating the solution?\n3. Are there any specific evaluation criteria or metrics that the evaluation agent will use to assess the consolidated solution?\n\nOnce I have a better understanding of these requirements, I'll begin the consolidation process and provide a step-by-step explanation of my thought process.\n\nPlease let me know if you'd like me to proceed with the consolidation, and I'll do my best to deliver a high-quality, comprehensive solution that addresses the user's needs and aligns with the prompt analysis.")]}
----
{'messages': [HumanMessage(content="''Please rate the following aspects of the proposed solution on a scale of 1 to 5, where:\n5 - Excellent\n4 - Good\n3 - Average\n2 - Poor\n1 - Unacceptable\n\n1. Accuracy: How accurate and precise is the solution in addressing your specific needs and requirements?\nUser Rating: 1\n\n2. Relevance: How relevant and applicable is the solution to your specific context, industry, or use case?\nUser Rating: 1\n\n3. Completeness: To what extent does the solution comprehensively address all aspects of your problem or need?\nUser Rating: 1")]}
----

----
''')

In [ ]:
agent_profiles['DataAnalysisAgent']["Tasks"]

In [ ]:
def print_dictionary_to_file(dictionary, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for key, value in dictionary.items():
            file.write(f"{key}: {value}\n")

# Example dictionary
my_dict = {
    "name": "John",
    "age": 30,
    "city": "New York"
}

# Specify the file path
file_path = r'C:\Users\crivera\OneDrive - NREL\code\LangGraph\dictionary_output.txt'

# Print out the dictionary to the file
print("Saving dictionary contents to file...")
print_dictionary_to_file(agent_profiles, file_path)
print("Dictionary contents saved to file successfully.")


## Test to see how the state is handed over and edited between nodes/agents

In [ ]:
def one(state):
    # messages = state["messages"]
    msg = "POE"
    return {"messages":msg}
    # return msg

def two(state):
    # messages = state["messages"]
    messages = " SECOND: B "
    return {"messages":messages}

def route(state):
    # messages = state["messages"][-1]
    # messages += " THIRD: C "
    # messages = " THIRD: C "

    # return {"messages":messages}
    return "POE"

def poe(state):
    # x = llm.invoke("hi :)").content
    x = "conditional passed"
    return {"messages": x}
test = StateGraph(AgentState)

test.add_node("1", one)
test.add_node("2", two)
test.add_node("POE", poe)
# test.add_node("3", three)
# test.add_node("4", four)

# need to add edges
test.add_edge("1","POE")

test.add_conditional_edges("1", route, {"POE":"POE"})

test.add_edge("POE","2")
# test.add_edge("2","3")


test.set_entry_point("1")
test.set_finish_point("2")

testing = test.compile()

In [ ]:
from langchain_core.messages import BaseMessage, HumanMessage

for s in testing.stream(
    {"messages": ""},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

In [ ]:
testing.get_graph().print_ascii()